In [7]:
import json
import os
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D  # Necessary for 3D plotting

def load_json_with_ids(json_filepath):
    """
    Loads point cloud data from a JSON file, mapping object names to vertex IDs and their coordinates.

    :param json_filepath: Path to the JSON file.
    :return: Dictionary mapping object names to another dictionary of vertex IDs and their coordinates.
    """
    if not os.path.exists(json_filepath):
        raise FileNotFoundError(f"JSON file not found at: {json_filepath}")

    with open(json_filepath, 'r') as f:
        data = json.load(f)

    objects = data.get("objects", [])
    point_clouds = {}
    for obj in objects:
        object_name = obj.get("object_name", "Unnamed_Object")
        vertices = obj.get("vertices", [])
        id_coord_map = {}
        for vertex in vertices:
            vid = vertex.get("id")
            coord = (vertex.get("x"), vertex.get("y"), vertex.get("z"))
            if vid is not None and None not in coord:
                id_coord_map[vid] = coord
        point_clouds[object_name] = id_coord_map

    return point_clouds

def compute_displacements(original_pcs, distorted_pcs):
    """
    Computes displacement vectors between original and distorted point clouds based on vertex IDs.

    :param original_pcs: Dictionary mapping object names to vertex IDs and their original coordinates.
    :param distorted_pcs: Dictionary mapping object names to vertex IDs and their distorted coordinates.
    :return: Dictionary mapping object names to another dictionary of vertex IDs and their displacement vectors.
    """
    displacement = {}
    common_objects = set(original_pcs.keys()).intersection(set(distorted_pcs.keys()))
    print(f"Number of common objects: {len(common_objects)}\n")

    for obj_name in common_objects:
        orig_vertices = original_pcs[obj_name]
        dist_vertices = distorted_pcs[obj_name]
        
        common_ids = set(orig_vertices.keys()).intersection(set(dist_vertices.keys()))
        if not common_ids:
            print(f"No common vertex IDs for object '{obj_name}'. Skipping.\n")
            continue

        displacement[obj_name] = {}
        for vid in common_ids:
            orig_coord = np.array(orig_vertices[vid])
            dist_coord = np.array(dist_vertices[vid])
            disp_vector = dist_coord - orig_coord
            displacement[obj_name][vid] = disp_vector

        print(f"Computed displacement vectors for object '{obj_name}': {len(displacement[obj_name])} vertices.\n")

    return displacement

def plot_displacement_vectors(original_pcs, displacement_vectors, sample_rate=1000, save_path=None):
    """
    Plots displacement vectors for each object in a 3D scatter plot.

    :param original_pcs: Dictionary mapping object names to vertex IDs and their original coordinates.
    :param displacement_vectors: Dictionary mapping object names to vertex IDs and their displacement vectors.
    :param sample_rate: Sampling rate for plotting (e.g., plot every 1000th vector).
    :param save_path: Optional path to save the plot image.
    """
    for obj_name, disp_dict in displacement_vectors.items():
        if not disp_dict:
            print(f"No displacement vectors to plot for object '{obj_name}'.\n")
            continue

        vids = list(disp_dict.keys())
        orig_coords = np.array([original_pcs[obj_name][vid] for vid in vids])
        disp_vectors = np.array([disp_dict[vid] for vid in vids])

        # Downsample
        if sample_rate > 0:
            orig_coords_sampled = orig_coords[::sample_rate]
            disp_vectors_sampled = disp_vectors[::sample_rate]
        else:
            orig_coords_sampled = orig_coords
            disp_vectors_sampled = disp_vectors

        # Create figure
        fig = plt.figure(figsize=(15, 10))
        ax = fig.add_subplot(111, projection='3d')

        # Plot original points
        ax.scatter(orig_coords_sampled[:,0], orig_coords_sampled[:,1], orig_coords_sampled[:,2],
                   c='blue', marker='o', s=10, label='Original Points')

        # Plot displacement vectors
        ax.quiver(orig_coords_sampled[:,0], orig_coords_sampled[:,1], orig_coords_sampled[:,2],
                  disp_vectors_sampled[:,0], disp_vectors_sampled[:,1], disp_vectors_sampled[:,2],
                  length=1.0, normalize=True, color='red', label='Displacement Vectors')

        # Setting labels and title
        ax.set_xlabel('X Coordinate')
        ax.set_ylabel('Y Coordinate')
        ax.set_zlabel('Z Coordinate')
        ax.set_title(f"Displacement Vectors for '{obj_name}' (Sample Rate: {sample_rate})")

        # Add legend
        ax.legend()

        # Adjust the aspect ratio
        max_range = np.array([orig_coords_sampled[:,0].max()-orig_coords_sampled[:,0].min(),
                              orig_coords_sampled[:,1].max()-orig_coords_sampled[:,1].min(),
                              orig_coords_sampled[:,2].max()-orig_coords_sampled[:,2].min()]).max() / 2.0

        mid_x = (orig_coords_sampled[:,0].max()+orig_coords_sampled[:,0].min()) * 0.5
        mid_y = (orig_coords_sampled[:,1].max()+orig_coords_sampled[:,1].min()) * 0.5
        mid_z = (orig_coords_sampled[:,2].max()+orig_coords_sampled[:,2].min()) * 0.5
        ax.set_xlim(mid_x - max_range, mid_x + max_range)
        ax.set_ylim(mid_y - max_range, mid_y + max_range)
        ax.set_zlim(mid_z - max_range, mid_z + max_range)

        # Save the plot if a save path is provided
        if save_path:
            if not os.path.exists(save_path):
                os.makedirs(save_path)
            plt.savefig(os.path.join(save_path, f"displacement_vectors_{obj_name}.png"), dpi=300)
            print(f"Saved displacement vector plot for '{obj_name}' to '{save_path}'.\n")

        plt.show()

def main():
    # Define paths to your JSON files
    original_json = "subdivided_objects_vertices_id.json"
    distorted_json = "subdivided_objects_vertices_dist_id.json"

    # Ensure the JSON files exist
    if not os.path.exists(original_json):
        print(f"Original JSON file '{original_json}' not found.")
        return
    if not os.path.exists(distorted_json):
        print(f"Distorted JSON file '{distorted_json}' not found.")
        return

    # Load the JSON data
    print("Loading original point cloud data...")
    original_pcs = load_json_with_ids(original_json)
    print("Loading distorted point cloud data...")
    distorted_pcs = load_json_with_ids(distorted_json)

    # Compute displacement vectors
    print("\nComputing displacement vectors...")
    displacement_vectors = compute_displacements(original_pcs, distorted_pcs)

    # Check if any displacement vectors were computed
    if not displacement_vectors:
        print("No displacement vectors were computed. Please check your JSON files for matching objects and vertex IDs.")
        return

    # Plot displacement vectors with a sample rate of 1000
    print("\nPlotting displacement vectors...")
    # You can adjust the sample_rate based on your data density
    # Lower sample_rate for more vectors, higher for fewer vectors
    plot_displacement_vectors(original_pcs, displacement_vectors, sample_rate=1000, save_path=None)
    print("\nVisualization completed.")



In [8]:
main()

Loading original point cloud data...
Loading distorted point cloud data...

Computing displacement vectors...
Number of common objects: 1

No common vertex IDs for object 'Circle.025'. Skipping.

No displacement vectors were computed. Please check your JSON files for matching objects and vertex IDs.
